In [11]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thur Apr 24

@author: yaning
"""

import importlib
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math

# my own class files
import weight_normalise.Receptors as Receptors
import weight_normalise.Network as Network
import weight_normalise.create_update_synapse as cus


importlib.reload(Receptors)
importlib.reload(Network)
importlib.reload(cus)

path = "/home/yaning/Documents/Spiking_NN/"
np.set_printoptions(threshold=np.inf)

In [12]:
with open(path + "datasets/email_scam.pkl", "rb") as f:
    fire_array, dataset = pickle.load(f)

In [13]:
fire_array.shape

(9, 4, 6000)

In [14]:
pointCount = 6000
deltaTms = 0.05
times = np.arange(pointCount) * deltaTms
initial_Vm = 1.3458754117369027

neuron_input_0 = Network.Neuron(deltaTms, 0, initial_Vm, "input_0")
neuron_input_1 = Network.Neuron(deltaTms, 0, initial_Vm, "input_1")

neuron_output_0 = Network.Neuron(deltaTms, 0, initial_Vm, "output_0")
neuron_output_1 = Network.Neuron(deltaTms, 0, initial_Vm, "output_1")

neurons = [neuron_input_0, neuron_input_1, neuron_output_0, neuron_output_1]
neuron_names = ["input_0", "input_1", "output_0", "output_1"]

cus.create_synapse(neuron_input_0, neuron_output_0, "AMPA")
cus.create_synapse(neuron_input_0, neuron_output_1, "AMPA")

cus.create_synapse(neuron_input_1, neuron_output_0, "AMPA")
cus.create_synapse(neuron_input_1, neuron_output_1, "AMPA")

In [15]:
for trial in range(fire_array.shape[0]):
    print("this is trial:", trial)
    fired_synapse = []
    for t in range(pointCount):
        if fire_array[trial, 0, t]:
            # print("this line runs")
            neuron_input_0.sending_signal()
            neuron_input_0.fire_tstep.append(t)
            
        if fire_array[trial, 1, t]:
            neuron_input_1.sending_signal()
            neuron_input_1.fire_tstep.append(t)
            

        # update the synapse states then each neuron
        # exclude checking the input neurons
        for neuron in neurons[2:]:
            if neuron.fire_tstep == []:
                last_fire = -2
            else:
                last_fire = neuron.fire_tstep[-1]
            fire = neuron.check_firing(t)
            if fire:
                # only update weight when it fires the first tstep
                if last_fire + 1 != t:
                    # neuron.update_weights_hebbian(t)
                    neuron.update_weights_twenty(t)

            
            neuron.update()


        
        
        # set the synapse states back to 0
        for synapse in cus.all_synapses:
            # if synapse.state == 1:
            #     # print("this line runs")
            #     fired_synapse.append(synapse)
            synapse.state = 0
        # print(fired_synapse)


    # desire = dataset[trial, -1]
    # print(len(fired_synapse))

    # difference = len(neuron_output_0.fire_tstep) - len(neuron_output_1.fire_tstep)
    # if  difference > 0:
    #     real = 0
    # else: real = 1
    # print(real)
    # # # rescaled_difference = sigmoid(difference)
    # if desire == real:
    #     for synapse in fired_synapse:
    #         for receptor in synapse.receptors:
    #             learning_weight_num += 1
    #             receptor.w = receptor.w * 1.1
    # else:
    #     for synapse in fired_synapse:
    #         for receptor in synapse.receptors:
    #             receptor.w = receptor.w * 0.9

this is trial: 0
this is trial: 1
this is trial: 2
this is trial: 3
this is trial: 4
this is trial: 5
this is trial: 6
this is trial: 7
this is trial: 8


In [16]:
for neuron in neurons:
    print(neuron.num)

0
0
49
24


In [17]:
for neuron in neurons:
    print(neuron.fire_tstep)

[3640, 3641, 3642, 3643, 3644, 3645, 3646, 3647, 3648, 3649, 3650, 3651, 3652, 3653, 3654, 3655, 3656, 3657, 3658, 3659, 3660, 3661, 3662, 3663, 3664, 3665, 3666, 3667, 3668, 3669, 3670, 3671, 3672, 3673, 3674, 3675, 3676, 3677, 3678, 3679, 3680, 3681, 3682, 3683, 3684, 3685, 3686, 3876, 3877, 3878, 3879, 3880, 3881, 3882, 3883, 3884, 3885, 3886, 3887, 3888, 3889, 3890, 3891, 3892, 3893, 3894, 3895, 3896, 3897, 3898, 3899, 3900, 3901, 3902, 3903, 3904, 3905, 3906, 3907, 3908, 3909, 3910, 3911, 3912, 3913, 3914, 3915, 3916, 3917, 3918, 3919, 3920, 3921, 3922, 4112, 4113, 4114, 4115, 4116, 4117, 4118, 4119, 4120, 4121, 4122, 4123, 4124, 4125, 4126, 4127, 4128, 4129, 4130, 4131, 4132, 4133, 4134, 4135, 4136, 4137, 4138, 4139, 4140, 4141, 4142, 4143, 4144, 4145, 4146, 4147, 4148, 4149, 4150, 4151, 4152, 4153, 4154, 4155, 4156, 4157, 4158, 4348, 4349, 4350, 4351, 4352, 4353, 4354, 4355, 4356, 4357, 4358, 4359, 4360, 4361, 4362, 4363, 4364, 4365, 4366, 4367, 4368, 4369, 4370, 4371, 4372, 437

In [18]:
for synapse in cus.all_synapses[:]:
    for receptor in synapse.receptors:
        print(receptor.w)

0.0217866461347168
3.0193089903603614
0.035219397170910015
3.8957032960891693


In [19]:
num_correct = 0

for trial in range(fire_array.shape[0]):
    print("this is trial", trial)

    for neuron in neurons:
        neuron.erase(initial_Vm)

    for t in range(pointCount):
        if fire_array[trial, 0, t]:
            neuron_input_0.sending_signal()
            neuron_input_0.fire_tstep.append(t)
            
        if fire_array[trial, 1, t]:
            neuron_input_1.sending_signal()
            neuron_input_1.fire_tstep.append(t)
            

        for neuron in neurons[2:]:
            if neuron.fire_tstep == []:
                last_fire = -2
            else:
                last_fire = neuron.fire_tstep[-1]
            fire = neuron.check_firing(t)
            
            neuron.update()

        # set the synapse states back to 0
        for synapse in cus.all_synapses:
            synapse.state = 0

    
    # if sum(output_synapse_states_0) - sum(output_synapse_states_1) > 0:
    if len(neuron_output_0.fire_tstep) - len(neuron_output_1.fire_tstep) > 0:
        print("predict 0 fires")
        predict = -1
    else:
        predict = 1
    if predict == dataset[trial][-1]:
        print("yes")
        num_correct += 1

this is trial 0
yes
this is trial 1
this is trial 2
yes
this is trial 3
this is trial 4
yes
this is trial 5
this is trial 6
this is trial 7
this is trial 8
yes


In [20]:
dataset

array([[ 5. , 10. ,  1. ],
       [ 1. ,  2. , -1. ],
       [ 7. , 12. ,  1. ],
       [ 0.5,  1. , -1. ],
       [ 4. ,  8. ,  1. ],
       [ 0.8,  0. , -1. ],
       [ 2. ,  1. , -1. ],
       [ 3. ,  0. , -1. ],
       [ 2. , 10. ,  1. ]])